In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query
import preprocesser
import datetime


def non_decreasing(L):
    data = [x<=y for x, y in zip(L, L[1:])]
    return data

In [2]:
#1.종목명
shcode_list = query.query_shcode_list_today()

#2.target from csv
df = pd.read_csv("high_ratio_0.15_volume_money_30000000000_v2.csv", dtype ={'shcode':str})

#3. target index 추출
fin_result = pd.DataFrame([],columns = ['target_day','shcode'])

for i,v in df['index'].items():
    shcode = df['shcode'].iloc[i].zfill(6)
    target_date = v.split(" ")
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=6)
    result = pd.DataFrame(dict(target_day = target_day_list.strftime("%Y%m%d"),
                              interval_end = interval_end_timestamp.strftime("%Y%m%d"))).reset_index()
    del result['index']
    result['shcode'] = shcode
    fin_result = fin_result.append(result)
#4. make target month
fin_result['target_m'] = fin_result['target_day'].str.slice(stop = 6)
fin_result = pd.merge(fin_result, shcode_list, how = 'inner', on = 'shcode')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [12]:
'''
#8월대상한정
#fin_result.groupby(['target_m']).count()
target_month = '201907'
fin_result = fin_result[fin_result['target_m'] == target_month]
fin_result = fin_result.reset_index()
#target_fin_result[['target_m','shcode','shname','target_day']].to_csv("201907.csv",encoding = "ms949")
'''

In [3]:
target_day_list = pd.to_datetime(fin_result['target_day'], format = "%Y-%m-%d")
#interval calculate before 6 month , after 2 month
init_timestamp = target_day_list - datetime.timedelta(weeks=5*30)
end_timestamp = target_day_list + datetime.timedelta(weeks=5*2)
fin_result['init_month'] = init_timestamp.apply(lambda x: x.strftime("%Y%m%d"))
fin_result['end_month'] = end_timestamp.apply(lambda x: x.strftime("%Y%m%d"))

In [4]:
fin_result.head()

,interval_end,shcode,target_day,target_m,shname,init_month,end_month
0,20190909,000140,20190729,201907,하이트진로홀딩스,20160912,20191007
1,20190626,000890,20190515,201905,보해양조,20160629,20190724
2,20190424,001140,20190313,201903,카리스국보,20160427,20190522
3,20191029,001340,20190917,201909,백광산업,20161101,20191126
4,20190619,001550,20190508,201905,조비,20160622,20190717


In [6]:
exx = pd.DataFrame([],columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'deal_f',
                                 'burn_ratio', 'rotate_ratio', 'for_net_buy', 'com_net_buy',
                                 'per_net_buy','index','shcode', 'shname', 'target_day', 'ma5_score','ma20_score','ma5_20_score'])

for i,v in fin_result['shcode'].items():
    df_monthly = query.query_t1305_month(shcode = fin_result['shcode'].iloc[i],
                                         fr = fin_result['init_month'].iloc[i],
                                         to = fin_result['end_month'].iloc[i])
    
    #except1: data없는경우
    if df_monthly.shape[0] ==0:
        continue
    # ma preprocesser
    df_monthly = preprocesser.sma_mothly(df_monthly)
    amount = [0.15, 0.15, 0.2, 0.2, 0.3]
    
    # 대세 판단1 : 월봉 5이평이 감소하지않음
    ma5 = non_decreasing(df_monthly['close_ma5'])[-5:]
    ma5_score = sum(x * y for x, y in zip(ma5, amount)) / sum(amount)
    
    # 대세 판단2 : 월봉 20이평이 감소하지않음
    ma20 = non_decreasing(df_monthly['close_ma20'])[-5:]
    ma20_score = sum(x * y for x, y in zip(ma20, amount)) / sum(amount)
    
    # 대세 판단3 : 월봉 ma5 - ma20  크눈중
    ma5_20 = non_decreasing(df_monthly['close_ma5']-df_monthly['close_ma20'])[-5:]
    ma5_20_score = sum(x * y for x, y in zip(ma5_20, amount)) / sum(amount)    
    
    df_monthly = df_monthly.tail(20)
    df_monthly['index'] = i
    df_monthly['shcode'] = fin_result['shcode'].iloc[i]
    df_monthly['shname'] = fin_result['shname'].iloc[i]
    df_monthly['target_day'] = fin_result['target_day'].iloc[i]
    
    df_monthly['ma5_score'] = ma5_score 
    df_monthly['ma20_score'] = ma20_score
    df_monthly['ma5_20_score'] = ma5_20_score
    
    exx = exx.append(df_monthly)
    print(i, v, '완료')
    

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


0 000140 완료
1 000890 완료
2 001140 완료
3 001340 완료
4 001550 완료
5 001550 완료
6 001745 완료
7 001820 완료
8 002070 완료
9 002070 완료
10 002140 완료
11 002140 완료
12 002140 완료
13 002150 완료
14 002210 완료
15 002210 완료
16 002320 완료
17 002360 완료
18 002450 완료
19 002700 완료
20 002720 완료
21 002780 완료
22 002780 완료
23 002995 완료
24 003010 완료
25 003495 완료
26 003560 완료
27 003850 완료
28 004060 완료
29 004270 완료
30 004720 완료
31 004770 완료
32 004830 완료
33 004870 완료
34 004980 완료
35 005110 완료
36 005110 완료
37 005320 완료
38 005360 완료
39 005390 완료
40 005420 완료
41 005690 완료
42 005740 완료
43 006090 완료
44 006890 완료
45 006980 완료
46 006980 완료
47 006980 완료
48 007110 완료
49 007110 완료
50 007210 완료
51 007460 완료
52 007540 완료
53 007540 완료
54 008040 완료
55 008350 완료
56 008420 완료
57 008700 완료
58 008730 완료
59 008775 완료
60 009290 완료
61 009320 완료
62 009320 완료
63 009410 완료
64 009410 완료
65 009680 완료
66 009680 완료
67 010100 완료
68 010100 완료
69 010420 완료
70 010660 완료
71 011150 완료
72 011150 완료
73 011150 완료
74 011280 완료
75 011280 완료
76 011390 완료
77 012160

In [7]:
exx[['index','shcode', 'shname', 'target_day', 'date', 'open', 'high', 'low', 'close', 'volume', 'deal_f',
     'burn_ratio', 'rotate_ratio', 'for_net_buy', 'com_net_buy',
     'per_net_buy', 'ma5_score','ma20_score','ma5_20_score']].to_csv("3_trend.csv",encoding = 'ms949')